## Introduction

`nonos` is a 2D visualization Python library, developed by Gaylor Wafflard-Fernandez (gaylor.wafflard@univ-grenoble-alpes.fr) initially for planet-disk numerical simulations, as well as a command line application. It works with VTK-formatted data from IDEFIX and PLUTO, and dat-formatted data from FARGO-ADSG and FARGO3D. Most of the implementation has been made for polar / spherical outputs, even though some basic operations are possible in cartesian.

In this tutorial, we will learn how to: 
- load an IDEFIX dataset with `nonos.api.GasDataSet`
- perform some operations on a field (e.g., slices, projection)
- create a new field with `nonos.api.compute`
- visualize the data

## Resources

See the [documentation](https://nonos.readthedocs.io/en/latest/) for an introduction of nonos.

## Visualization from an Idefix point of view

### 1 - Loading a dataset

Get the path of the directory where you ran the planet-disk interaction problem (`../AdvancedSetup/problem1/`), and import the following libraries (in particular `GasDataSet` from `nonos.api`)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
from nonos.api import GasDataSet
from nonos.styling import set_mpl_style

set_mpl_style(1)  # not mandatory, just a custom style

In [ ]:
directory = "../../AdvancedSetup/solution1/"  # CHECK YOUR PATH!
# load the VTK file data.0003.vtk in the dataset ds
ds = GasDataSet(3, directory=directory)

`ds` is a field container, and you can access the fields in the form of a dictionary. You can check what fields are included in `ds` by running:

In [ ]:
list(ds.keys())

In [ ]:
ds, ds["RHO"]

### 2 - Access the data and perform operations

For this dataset, the simulation is in polar coordinates ($R$, $\phi$, $z$):

In [ ]:
print(f"{ds.native_geometry}: {ds.coords.cube}")

You can therefore have access to a 1D array of the coordinates in the simulation with `ds.coords.R`, `ds.coords.phi` and `ds.coords.z` for the cell edges and `ds.coords.Rmed`, `ds.coords.phimed`, `ds.coords.zmed` for the cell centers.

In [ ]:
ds.coords.phi

Concerning the data for the different fields including in the vtk, e.g. the density field `ds["RHO"]`, you can access it using `ds["RHO"].data`, which is basically a numpy array. You can therefore have access to useful information such as its shape:

In [ ]:
ds["RHO"].data.shape

Several operations can be made on the fields depending on the geometry, for example here you can perform an azimuthal average of the density.

In [ ]:
# Azimuhtally averaged density field
ds["RHO"].azimuthal_average()

As you can see, it creates a new field with the same attributes as the original density field, but with a reduced dimension.

In [ ]:
ds["RHO"].azimuthal_average().data.shape

The general operations that are available depending on the native geometry of the data are listed in the documentation on [this link](https://nonos.readthedocs.io/en/latest/api/#1-general-operations) (example: `radial_at_r(distance)` for a slice at a given radius, or `latitudinal_projection(theta)` for an integral between $-\theta$ and $\theta$)

### 3 - Compute a new field

We now want to consider the azimuthal component of the gas velocity ($v_\phi$) computed in the frame corotating with the planet  (located in $R_p$, rotating at $\Omega_p=R_p^{-3/2}$), and therefore create a corrotating field $v_\phi^C$

$v_\phi^C = v_\phi - R\Omega_p$

We start by definiting the planet's parameters

In [ ]:
# radial location of the planet
Rp = 1.0
# Angular velocity of the planet
OmegaP = Rp**(-3/2)

In order to compute a new field from preexisting ones, you can use the `compute` function imported from `nonos.api`, which takes 3 arguments:
- the name of the new field
- the corresponding array
- a known field with similar structure as the new field 

In [ ]:
from nonos.api import compute

# 3D array of the radial coordinate
Rmed3d = ds.coords.Rmed[:, None, None]

vphifield = compute(
    field="VPHI",
    data=ds["VX2"].data - Rmed3d * (OmegaP),
    ref=ds["VX2"],  # same structure as v_phi field
)

### 4. Visualize the data

Before plotting the data, we need to convert the field into an object that can be plotted. To do that, nonos will map the field from the native geometry of the outputs to a target geometry. For example, if the native geometry of the dataset is `polar` ($R$, $\phi$, $z$), you can map it (and then visualize it) in a polar plane ($R$, $\phi$), but also in a cartesian plane ($x$, $y$).

In [ ]:
ds["RHO"].map("x", "y")

Note that here there is just one cell in the vertical direction, so you don't need to perform a vertical slice beforehand. But in a general 3D case, you need first to reduce the dimension with one or two operations, and then map the field in the target plane.

In [ ]:
# Create figure and subplots with matplotlib.pyplot
fig, ax = plt.subplots()
# Density field mapped in the cartesian plane (x, y), and then plotted in log scale
ds["RHO"].map("x", "y").plot(fig, ax, log=True, cmap="inferno", title=r"$\rho$")
# Same scaling for x-axis and y-axis
ax.set(aspect="equal")
plt.show()

We have here a cartesian view of the density field, in log scale, using the colormap `"inferno"` and with a colorbar and title. We can do the same in a polar view.

In [ ]:
fig, ax = plt.subplots()
# Density field mapped in the polar plane (phi, R), and then plotted in log scale
ds["RHO"].map("phi", "R").plot(
    fig, ax, log=True, cmap="inferno", title=r"$\rho$"
)
plt.show()

In [ ]:
# Same here, but reversing the axes (phi, R) -> (R, phi)
# and superimposing other plots (vertical lines)
fig, ax = plt.subplots()
ds["RHO"].map("R", "phi").plot(fig, ax, log=True, cmap="inferno", title=r"$\rho$")
for rad, lsi in zip(np.linspace(1.1, 2.0, 3), ["-", "--", "-.", ":"]):
    ax.axvline(x=rad, c="k", ls=lsi, label=f"R = {rad:.1f}")
ax.legend(frameon=False, loc=2)
plt.show()

Here we added some vertical lines at different radial locations, and we can perform various slices at these locations in order to visualize the density field along these lines.

In [ ]:
fig, ax = plt.subplots()
for rad, lsi in zip(np.linspace(1.1, 2.0, 3), ["-", "--", "-.", ":"]):
    ds["RHO"].radial_at_r(rad).map("phi").plot(
        fig, ax, log=True, title=r"$\rho$", c="k", ls=lsi, label=f"R = {rad:.1f}"
    )
ax.set(ylim=(1.4, 2.5), xlabel=r"$\phi$")
plt.legend(frameon=False)
plt.show()

### 5. Bonus: Line Integral Convolution (LIC)

Here, we introduce the class `NonosLick` (not directly in the main library), and we can now use various functions and arguments that we introduced earlier, like `compute` or the coordinates at the cell center `ds.coords.Rmed`, `ds.coords.phimed`, and perform a Line Integral Convolution (method to visualize a vector field).

#### 5.1 Get the coordinates

In the next cell, we first convert from polar to cartesian the coordinates $(R, \phi) \rightarrow (x, y)$:

$x = R \cos(\phi)$ \
$y = R \sin(\phi)$

In [ ]:
from nonos.api import NonosLick

# Radial and azimuthal coordinates
Rmed = ds.coords.Rmed
phimed = ds.coords.phimed
# 3D array of the azimuthal coordinate
pmed3d = phimed[None, :, None]

# Return a list of coordinate matrices from coordinate vectors.
RRmed, ppmed = np.meshgrid(Rmed, phimed, indexing="ij")
# Convert from polar to cartesian
xxmed = RRmed * np.cos(ppmed)
yymed = RRmed * np.sin(ppmed)

#### 5.2 Compute the data

Then we convert from polar to cartesian the the data $(v_R, v_\phi^C) \rightarrow (v_x, v_y)$:

$v_x = v_R \cos(\phi) - v_\phi^C \sin(\phi)$ \
$v_y = v_R \sin(\phi) + v_\phi^C \cos(\phi)$

In [ ]:
# Compute v_x from (v_R, v_phi^C)
vxfield = compute(
    field="VX",
    data=ds["VX1"].data * np.cos(pmed3d) - vphifield.data * np.sin(pmed3d),
    ref=ds["VX1"],
)

# Compute v_y from (v_R, v_phi^C)
vyfield = compute(
    field="VY",
    data=ds["VX1"].data * np.sin(pmed3d) + vphifield.data * np.cos(pmed3d),
    ref=ds["VX1"],
)

#### 5.3 Perform the LIC

Then, we perform a LIC in a specific area around the planet:

In [ ]:
# Define the area where we compute the LIC, and eventually the zoom of the final plot
xmin, xmax, ymin, ymax = (0.5, 1.5, -0.5, 0.5)

# LIC in cartesian, with the density field as background
lick = NonosLick(
    xxmed,
    yymed,
    vxfield,
    vyfield,
    ds["RHO"],
    xmin=xmin,
    xmax=xmax,
    ymin=ymin,
    ymax=ymax,
)

#### 5.4 Plot the LIC

Finally, we plot the LIC with some streamlines, demonstrating the flow around the planet, with the "horseshoe" coorbital regions clearly visible.

In [ ]:
fig, ax = plt.subplots(figsize=(6, 8))
# Plot the density + LIC for the velocity vector
# with some streamlines. We are in corotation with the planet
lick.plot(fig, ax, log=True, title=r"$\rho$", density_streamlines=1, cmap="inferno")
ax.set(aspect="equal", xlim=(xmin, xmax), ylim=(ymin, ymax))
plt.show()